<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="https://www.uoc.edu/content/dam/news/images/noticies/2016/202-nova-marca-uoc.jpg" align="left" width="45%">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">Trabajo de Fin de Máster</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Machine Learning para predecir cancelaciones y mejorar la retención en seguros

## Optimización

In [2]:
import pandas as pd
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv').squeeze()
X_train_ros = pd.read_csv('X_train_ros.csv')
y_train_ros = pd.read_csv('y_train_ros.csv').squeeze()
X_train_rus = pd.read_csv('X_train_rus.csv')
y_train_rus = pd.read_csv('y_train_rus.csv').squeeze()
X_train_smote = pd.read_csv('X_train_smote.csv')
y_train_smote = pd.read_csv('y_train_smote.csv').squeeze()

In [3]:
from sklearn.metrics import make_scorer, f1_score, roc_auc_score, classification_report
from sklearn.model_selection import GridSearchCV

In [27]:
from sklearn.naive_bayes import GaussianNB

# Definimos el espacio de búsqueda para var_smoothing
param_grid = {
    'var_smoothing': [4e-1, 5e-1, 6e-1,7e-1, 8e-1, 9e-1,1 ,2, 10, 50, 100, 110, 115]
}

# Métrica principal a optimizar
scorer = make_scorer(f1_score)

# Configuramos la búsqueda en cuadrícula
grid_search = GridSearchCV(
    estimator=GaussianNB(),
    param_grid=param_grid,
    scoring=scorer,
    cv=2,
    n_jobs=-1,
    verbose=1
)

# Ajustamos sobre los datos submuestreados
grid_search.fit(X_train_rus, y_train_rus)

# Extraemos el mejor modelo
best_nb = grid_search.best_estimator_

# Predicciones con el conjunto de test original
y_pred_rus = best_nb.predict(X_test)
y_proba_rus = best_nb.predict_proba(X_test)[:, 1]

# Evaluación
print("\nResultados para RandomUnderSampler (GaussianNB) con Hypertuning:")
print(f"Mejor hiperparámetro (var_smoothing): {grid_search.best_params_}")
print(f"AUC: {roc_auc_score(y_test, y_proba_rus):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_rus):.4f}")
print(classification_report(y_test, y_pred_rus))


Fitting 2 folds for each of 13 candidates, totalling 26 fits

Resultados para RandomUnderSampler (GaussianNB) con Hypertuning:
Mejor hiperparámetro (var_smoothing): {'var_smoothing': 110}
AUC: 0.6802
F1-score: 0.2725
              precision    recall  f1-score   support

           0       0.93      0.60      0.73    422179
           1       0.17      0.65      0.27     54408

    accuracy                           0.60    476587
   macro avg       0.55      0.62      0.50    476587
weighted avg       0.84      0.60      0.67    476587



In [14]:
from sklearn.neighbors import KNeighborsClassifier

# Definimos el espacio de búsqueda para los hiperparámetros
param_grid = {
    'n_neighbors': [10, 15, 20],
    'weights': ['uniform', 'distance'],
}

# Definimos la métrica principal para optimizar (F1 en este caso)
scorer = make_scorer(f1_score)

# Configuramos la búsqueda en cuadrícula
grid_search = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid,
    scoring=scorer,
    cv=2,
    n_jobs=-1,
    verbose=1
)

# Ejecutamos el ajuste sobre los datos submuestreados
grid_search.fit(X_train_rus, y_train_rus)

# Extraemos el mejor modelo
best_knn = grid_search.best_estimator_

# Predicciones con el conjunto de test original
y_pred_rus = best_knn.predict(X_test)
y_proba_rus = best_knn.predict_proba(X_test)[:, 1]

# Evaluación
print("\nResultados para RandomUnderSampler (KNN) con Hypertuning:")
print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"AUC: {roc_auc_score(y_test, y_proba_rus):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_rus):.4f}")
print(classification_report(y_test, y_pred_rus))


Fitting 2 folds for each of 6 candidates, totalling 12 fits

Resultados para RandomUnderSampler (KNN) con Hypertuning:
Mejores hiperparámetros: {'n_neighbors': 20, 'weights': 'distance'}
AUC: 0.6798
F1-score: 0.2994
              precision    recall  f1-score   support

           0       0.93      0.70      0.80    422179
           1       0.20      0.58      0.30     54408

    accuracy                           0.69    476587
   macro avg       0.57      0.64      0.55    476587
weighted avg       0.85      0.69      0.74    476587



In [15]:
from sklearn.tree import DecisionTreeClassifier

# Definimos el espacio de búsqueda
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5, 10],
}

# Métrica objetivo: F1
scorer = make_scorer(f1_score)

# Búsqueda con validación cruzada
grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid=param_grid,
    scoring=scorer,
    cv=2,
    n_jobs=-1,
    verbose=1
)

# Ajuste con datos SMOTE
grid_search.fit(X_train_smote, y_train_smote)

# Mejor modelo
best_dt = grid_search.best_estimator_

# Predicción en test original
y_pred_smote = best_dt.predict(X_test)
y_proba_smote = best_dt.predict_proba(X_test)[:, 1]

# Evaluación
print("\nResultados para SMOTE (Decision Tree) con Hypertuning:")
print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"AUC: {roc_auc_score(y_test, y_proba_smote):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_smote):.4f}")
print(classification_report(y_test, y_pred_smote))


Fitting 2 folds for each of 18 candidates, totalling 36 fits

Resultados para SMOTE (Decision Tree) con Hypertuning:
Mejores hiperparámetros: {'criterion': 'gini', 'max_depth': 20, 'min_samples_split': 2}
AUC: 0.6524
F1-score: 0.3963
              precision    recall  f1-score   support

           0       0.92      0.95      0.93    422179
           1       0.46      0.35      0.40     54408

    accuracy                           0.88    476587
   macro avg       0.69      0.65      0.66    476587
weighted avg       0.87      0.88      0.87    476587



In [16]:
from sklearn.ensemble import RandomForestClassifier

# Definimos el espacio de búsqueda para los hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 4],
}

# Definimos el scorer para optimizar el F1-score
scorer = make_scorer(f1_score)

# Configuramos la búsqueda en cuadrícula
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    scoring=scorer,
    cv=2,
    n_jobs=-1,
    verbose=1
)

# Ajustamos el modelo con los datos de SMOTE
grid_search.fit(X_train_smote, y_train_smote)

# Extraemos el mejor modelo
best_rf = grid_search.best_estimator_

# Predicciones sobre el conjunto de prueba
y_pred_smote = best_rf.predict(X_test)
y_proba_smote = best_rf.predict_proba(X_test)[:, 1]

# Evaluación
print("\nResultados para SMOTE (Random Forest) con Hypertuning:")
print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"AUC: {roc_auc_score(y_test, y_proba_smote):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_smote):.4f}")
print(classification_report(y_test, y_pred_smote))


Fitting 2 folds for each of 36 candidates, totalling 72 fits

Resultados para SMOTE (Random Forest) con Hypertuning:
Mejores hiperparámetros: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}
AUC: 0.6937
F1-score: 0.4533
              precision    recall  f1-score   support

           0       0.93      0.93      0.93    422179
           1       0.46      0.45      0.45     54408

    accuracy                           0.88    476587
   macro avg       0.69      0.69      0.69    476587
weighted avg       0.88      0.88      0.88    476587



In [17]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, classification_report

# Definimos el espacio de búsqueda para los hiperparámetros
param_grid = {
    'n_estimators': [150, 200, 250],
    'learning_rate': [0.01, 0.1, 0.5, 1],
}

# Configuramos la búsqueda en cuadrícula
grid_search = GridSearchCV(
    estimator=AdaBoostClassifier(random_state=42),
    param_grid=param_grid,
    scoring='f1',  # Utilizamos F1-score como métrica
    cv=2,
    n_jobs=-1,
    verbose=1
)

# Ajustamos el modelo con los datos sobre muestreo RandomOverSampler
grid_search.fit(X_train_ros, y_train_ros)

# Extraemos el mejor modelo
best_ada = grid_search.best_estimator_

# Predicciones sobre el conjunto de prueba
y_pred_ros = best_ada.predict(X_test)
y_proba_ros = best_ada.predict_proba(X_test)[:, 1]

# Evaluación
print("\nResultados para RandomOverSampler (AdaBoost) con Hypertuning:")
print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"AUC: {roc_auc_score(y_test, y_proba_ros):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_ros):.4f}")
print(classification_report(y_test, y_pred_ros))

Fitting 2 folds for each of 12 candidates, totalling 24 fits

Resultados para RandomOverSampler (AdaBoost) con Hypertuning:
Mejores hiperparámetros: {'learning_rate': 1, 'n_estimators': 250}
AUC: 0.6946
F1-score: 0.4114
              precision    recall  f1-score   support

           0       0.93      0.89      0.91    422179
           1       0.36      0.48      0.41     54408

    accuracy                           0.84    476587
   macro avg       0.65      0.68      0.66    476587
weighted avg       0.86      0.84      0.85    476587



In [28]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, classification_report

# Definimos el espacio de búsqueda para los hiperparámetros
param_grid = {
    'n_estimators': [300, 400, 500],
    'learning_rate': [0.001, 0.1, 0.5, 0.82],
    'max_depth': [30, 40, 50, 60],
    'gamma': [0, 0.1, 0.2]
}

# Configuramos la búsqueda en cuadrícula
grid_search = GridSearchCV(
    estimator=XGBClassifier(eval_metric='logloss', random_state=42, n_jobs=-1),
    param_grid=param_grid,
    scoring='f1',  # Utilizamos F1-score como métrica
    cv=2,
    n_jobs=-1,
    verbose=1
)

# Ajustamos el modelo con los datos sobre muestreo RandomOverSampler
grid_search.fit(X_train_ros, y_train_ros)

# Extraemos el mejor modelo
best_xgb = grid_search.best_estimator_

# Predicciones sobre el conjunto de prueba
y_pred_ros = best_xgb.predict(X_test)
y_proba_ros = best_xgb.predict_proba(X_test)[:, 1]

# Evaluación
print("\nResultados para RandomOverSampler (XGBoost) con Hypertuning:")
print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"AUC: {roc_auc_score(y_test, y_proba_ros):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_ros):.4f}")
print(classification_report(y_test, y_pred_ros))

Fitting 2 folds for each of 144 candidates, totalling 288 fits


/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
python(2722) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(2957) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(3031) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(4135) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(4228) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(5210) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(7435) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(7682) MallocStackLogging: can't turn off malloc stack logging 

KeyboardInterrupt: 

In [5]:
import numpy as np
import tensorflow as tf
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# División estratificada para validación
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_smote, y_train_smote, test_size=0.2, random_state=42, stratify=y_train_smote
)

def objective(trial):
    # Arquitectura de red
    model = Sequential([
        Input(shape=(X_train_final.shape[1],)),
        Dense(trial.suggest_int('units1', 64, 256), activation='relu'),
        Dropout(trial.suggest_float('dropout1', 0.2, 0.6)),
        Dense(trial.suggest_int('units2', 32, 128), activation='relu'),
        Dropout(trial.suggest_float('dropout2', 0.1, 0.4)),
        Dense(1, activation='sigmoid')
    ])
    
    # Optimización de hiperparámetros
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=trial.suggest_float('lr', 1e-4, 1e-2)
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Entrenamiento con early stopping
    history = model.fit(
        X_train_final, y_train_final,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=trial.suggest_categorical('batch_size', [64, 128, 256]),
        verbose=0,
        callbacks=[EarlyStopping(patience=10, restore_best_weights=True)]
    )
    
    # Cálculo de F1 score
    y_pred = (model.predict(X_val) > 0.5).astype(int)
    return f1_score(y_val, y_pred)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2025-05-06 04:43:28,334] A new study created in memory with name: no-name-19a82484-c0f0-41e0-873f-1015a930d84d


12314/12314 ━━━━━━━━━━━━━━━━━━━━ 3s 220us/step


[I 2025-05-06 05:01:22,823] Trial 0 finished with value: 0.6425449061790138 and parameters: {'units1': 250, 'dropout1': 0.2638138735924608, 'units2': 105, 'dropout2': 0.20603272797717542, 'lr': 0.00010616166714444989, 'batch_size': 128}. Best is trial 0 with value: 0.6425449061790138.


12314/12314 ━━━━━━━━━━━━━━━━━━━━ 3s 216us/step


[I 2025-05-06 05:05:01,726] Trial 1 finished with value: 0.6221388343701675 and parameters: {'units1': 226, 'dropout1': 0.4451994247393244, 'units2': 38, 'dropout2': 0.18472860447750317, 'lr': 0.006541503635203153, 'batch_size': 256}. Best is trial 0 with value: 0.6425449061790138.


...

12314/12314 ━━━━━━━━━━━━━━━━━━━━ 3s 213us/step


[I 2025-05-06 08:42:22,400] Trial 24 finished with value: 0.629511473267479 and parameters: {'units1': 166, 'dropout1': 0.23090130396833522, 'units2': 114, 'dropout2': 0.12246956123879757, 'lr': 0.0033314744013472055, 'batch_size': 128}. Best is trial 12 with value: 0.6887561824949536.


12314/12314 ━━━━━━━━━━━━━━━━━━━━ 3s 208us/step


[I 2025-05-06 08:53:31,229] Trial 25 finished with value: 0.6266008026379692 and parameters: {'units1': 117, 'dropout1': 0.34608812546844925, 'units2': 88, 'dropout2': 0.22955462228969054, 'lr': 0.0018212080528347279, 'batch_size': 128}. Best is trial 12 with value: 0.6887561824949536.
[W 2025-05-06 09:05:11,687] Trial 26 failed with parameters: {'units1': 181, 'dropout1': 0.20180740177426942, 'units2': 77, 'dropout2': 0.1307805246966578, 'lr': 0.0009135089099540973, 'batch_size': 128} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/luisocanahober/anaconda3/lib/python3.11/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/c9/t4s6bts95ybb23rnpkn7kc100000gn/T/ipykernel_2241/1178642045.py", line 35, in objective
    history = model.fit(
              ^^^^^^^^^^
  File "/Users/luisocanahober/anaconda3/lib/python3.11/site-packages/ker

KeyboardInterrupt: 

In [6]:
# Recuperar los mejores hiperparámetros
best_params = {
    'units1': 117,
    'dropout1': 0.34608812546844925,
    'units2': 88,
    'dropout2': 0.22955462228969054,
    'lr': 0.0018212080528347279,
    'batch_size': 128
}

# Reconstruir el modelo con los hiperparámetros óptimos
final_model = Sequential([
    Input(shape=(X_train_smote.shape[1],)),
    Dense(best_params['units1'], activation='relu'),
    Dropout(best_params['dropout1']),
    Dense(best_params['units2'], activation='relu'),
    Dropout(best_params['dropout2']),
    Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=best_params['lr'])
final_model.compile(optimizer=optimizer,
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

# Ajuste del modelo
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
final_model.fit(
    X_train_final, y_train_final,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=best_params['batch_size'],
    callbacks=[early_stop],
    verbose=0
)

from sklearn.metrics import roc_auc_score, f1_score, classification_report

# Predicciones finales sobre el conjunto de prueba
y_pred_test = (final_model.predict(X_test) > 0.5).astype(int).flatten()
y_proba_test = final_model.predict(X_test).flatten()

# Métricas de evaluación
print("\nResultados para el modelo neuronal (Optuna + SMOTE):")
print(f"Mejores hiperparámetros: {best_params}")
print(f"AUC: {roc_auc_score(y_test, y_proba_test):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_test):.4f}")
print(classification_report(y_test, y_pred_test))

14894/14894 ━━━━━━━━━━━━━━━━━━━━ 3s 205us/step
14894/14894 ━━━━━━━━━━━━━━━━━━━━ 3s 205us/step

Resultados para el modelo neuronal (Optuna + SMOTE):
Mejores hiperparámetros: {'units1': 117, 'dropout1': 0.34608812546844925, 'units2': 88, 'dropout2': 0.22955462228969054, 'lr': 0.0018212080528347279, 'batch_size': 128}
AUC: 0.6923
F1-score: 0.4302
              precision    recall  f1-score   support

           0       0.93      0.91      0.92    422179
           1       0.40      0.46      0.43     54408

    accuracy                           0.86    476587
   macro avg       0.67      0.69      0.68    476587
weighted avg       0.87      0.86      0.86    476587



In [8]:
import optuna
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import f1_score

# Definir la función de objetivo para Optuna
def objective(trial):
    # Definir el espacio de búsqueda de hiperparámetros
    n_neurons_1 = trial.suggest_int('n_neurons_1', 32, 128)  # Neuronas en la primera capa
    n_neurons_2 = trial.suggest_int('n_neurons_2', 16, 64)   # Neuronas en la segunda capa
    dropout_1 = trial.suggest_float('dropout_1', 0.1, 0.5)    # Tasa de Dropout en la primera capa
    dropout_2 = trial.suggest_float('dropout_2', 0.1, 0.5)    # Tasa de Dropout en la segunda capa
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)  # Tasa de aprendizaje

    # Crear el modelo
    model = Sequential([
        Input(shape=(X_train_ros.shape[1],)),
        Dense(n_neurons_1, activation='relu'),
        BatchNormalization(),
        Dropout(dropout_1),
        Dense(n_neurons_2, activation='relu'),
        BatchNormalization(),
        Dropout(dropout_2),
        Dense(1, activation='sigmoid')
    ])

    # Compilar el modelo con la tasa de aprendizaje elegida
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

    # Definir los callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

    # Dividir los datos para entrenamiento y validación
    X_ros_train, X_ros_val, y_ros_train, y_ros_val = train_test_split(
        X_train_ros, y_train_ros, test_size=0.2, stratify=y_train_ros, random_state=42
    )

    # Entrenamiento del modelo
    history = model.fit(X_ros_train, y_ros_train,
                        validation_data=(X_ros_val, y_ros_val),
                        epochs=100,
                        batch_size=256,
                        callbacks=[early_stop, reduce_lr],
                        verbose=0)

    # Evaluar el modelo con F1-score
    y_pred_ros_nn = model.predict(X_ros_val).ravel()
    y_pred_class = (y_pred_ros_nn >= 0.5).astype(int)  # Convertir probabilidades en predicciones binarias

    f1 = f1_score(y_ros_val, y_pred_class)
    
    # Se retorna el F1-score como métrica de optimización
    return f1

In [9]:
# Crear el estudio de Optuna
study = optuna.create_study(direction='maximize')  # Queremos maximizar el F1-score
study.optimize(objective, n_trials=10)  # Realiza 10 intentos

# Mostrar los mejores hiperparámetros
print(f"Mejores hiperparámetros: {study.best_params}")

[I 2025-05-14 21:57:20,622] A new study created in memory with name: no-name-36b8ed2e-f65c-4b04-a1eb-94a7aa6d6013
/var/folders/5w/3ld8z599699cd303197k9s9r0000gn/T/ipykernel_10680/52109843.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)  # Tasa de aprendizaje



Epoch 26: ReduceLROnPlateau reducing learning rate to 0.0004060380626469851.

Epoch 37: ReduceLROnPlateau reducing learning rate to 8.120761485770346e-05.

Epoch 44: ReduceLROnPlateau reducing learning rate to 1.62415235536173e-05.

Epoch 49: ReduceLROnPlateau reducing learning rate to 3.24830471072346e-06.
12314/12314 ━━━━━━━━━━━━━━━━━━━━ 2s 156us/step


[I 2025-05-14 22:00:46,508] Trial 0 finished with value: 0.6001143503019046 and parameters: {'n_neurons_1': 66, 'n_neurons_2': 50, 'dropout_1': 0.30444220193205873, 'dropout_2': 0.14072033584312837, 'learning_rate': 0.0020301903765699352}. Best is trial 0 with value: 0.6001143503019046.
/var/folders/5w/3ld8z599699cd303197k9s9r0000gn/T/ipykernel_10680/52109843.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)  # Tasa de aprendizaje



Epoch 37: ReduceLROnPlateau reducing learning rate to 2.3777037858963014e-05.

Epoch 43: ReduceLROnPlateau reducing learning rate to 4.755407644552179e-06.

Epoch 48: ReduceLROnPlateau reducing learning rate to 1e-06.
12314/12314 ━━━━━━━━━━━━━━━━━━━━ 2s 168us/step


[I 2025-05-14 22:04:00,544] Trial 1 finished with value: 0.6003167031335213 and parameters: {'n_neurons_1': 46, 'n_neurons_2': 57, 'dropout_1': 0.433285768290593, 'dropout_2': 0.447831136640483, 'learning_rate': 0.0001188851883982773}. Best is trial 1 with value: 0.6003167031335213.
/var/folders/5w/3ld8z599699cd303197k9s9r0000gn/T/ipykernel_10680/52109843.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)  # Tasa de aprendizaje


...


Epoch 96: ReduceLROnPlateau reducing learning rate to 5.621584205073305e-06.
12314/12314 ━━━━━━━━━━━━━━━━━━━━ 2s 152us/step


[I 2025-05-14 22:45:09,507] Trial 8 finished with value: 0.5999334622365253 and parameters: {'n_neurons_1': 72, 'n_neurons_2': 21, 'dropout_1': 0.1600627802915284, 'dropout_2': 0.265430451847538, 'learning_rate': 2.810792065892616e-05}. Best is trial 1 with value: 0.6003167031335213.
/var/folders/5w/3ld8z599699cd303197k9s9r0000gn/T/ipykernel_10680/52109843.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)  # Tasa de aprendizaje



Epoch 26: ReduceLROnPlateau reducing learning rate to 0.00010319027351215483.

Epoch 38: ReduceLROnPlateau reducing learning rate to 2.063805441139266e-05.

Epoch 44: ReduceLROnPlateau reducing learning rate to 4.1276107367593795e-06.

Epoch 49: ReduceLROnPlateau reducing learning rate to 1e-06.
12314/12314 ━━━━━━━━━━━━━━━━━━━━ 2s 156us/step


[I 2025-05-14 22:49:35,994] Trial 9 finished with value: 0.5990216399966836 and parameters: {'n_neurons_1': 63, 'n_neurons_2': 53, 'dropout_1': 0.16779733045276513, 'dropout_2': 0.2808276788160956, 'learning_rate': 0.0005159513634286087}. Best is trial 1 with value: 0.6003167031335213.


Mejores hiperparámetros: {'n_neurons_1': 46, 'n_neurons_2': 57, 'dropout_1': 0.433285768290593, 'dropout_2': 0.447831136640483, 'learning_rate': 0.0001188851883982773}


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

X_ros_train, X_ros_val, y_ros_train, y_ros_val = train_test_split(
        X_train_ros, y_train_ros, test_size=0.2, stratify=y_train_ros, random_state=42
    )

# Mejores hiperparámetros encontrados por Optuna
best_params = {
    'n_neurons_1': 46,
    'n_neurons_2': 57,
    'dropout_1': 0.433285768290593,
    'dropout_2': 0.447831136640483,
    'learning_rate': 0.0001188851883982773
}

# Reconstrucción del modelo
final_model = Sequential([
    Input(shape=(X_train_ros.shape[1],)),
    Dense(best_params['n_neurons_1'], activation='relu'),
    BatchNormalization(),
    Dropout(best_params['dropout_1']),
    Dense(best_params['n_neurons_2'], activation='relu'),
    BatchNormalization(),
    Dropout(best_params['dropout_2']),
    Dense(1, activation='sigmoid')
])

# Compilación
optimizer = Adam(learning_rate=best_params['learning_rate'])
final_model.compile(optimizer=optimizer,
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

# Entrenamiento
final_model.fit(
    X_train_ros, y_train_ros,
    validation_data=(X_ros_val, y_ros_val),
    epochs=100,
    batch_size=256,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# Evaluación
y_pred_test = (final_model.predict(X_test) > 0.5).astype(int).flatten()
y_proba_test = final_model.predict(X_test).flatten()

print("\nResultados para el modelo neuronal (Optuna + RandomOverSampler):")
print(f"Mejores hiperparámetros: {best_params}")
print(f"AUC: {roc_auc_score(y_test, y_proba_test):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_test):.4f}")
print(classification_report(y_test, y_pred_test))


Epoch 1/100
7696/7696 ━━━━━━━━━━━━━━━━━━━━ 5s 590us/step - accuracy: 0.5962 - loss: 0.7353 - val_accuracy: 0.6759 - val_loss: 0.6041 - learning_rate: 1.1889e-04
Epoch 2/100
7696/7696 ━━━━━━━━━━━━━━━━━━━━ 4s 558us/step - accuracy: 0.6756 - loss: 0.6124 - val_accuracy: 0.6858 - val_loss: 0.5951 - learning_rate: 1.1889e-04
Epoch 3/100
7696/7696 ━━━━━━━━━━━━━━━━━━━━ 5s 603us/step - accuracy: 0.6841 - loss: 0.6027 - val_accuracy: 0.6870 - val_loss: 0.5933 - learning_rate: 1.1889e-04
Epoch 4/100
7696/7696 ━━━━━━━━━━━━━━━━━━━━ 5s 605us/step - accuracy: 0.6855 - loss: 0.5994 - val_accuracy: 0.6876 - val_loss: 0.5924 - learning_rate: 1.1889e-04
Epoch 5/100
7696/7696 ━━━━━━━━━━━━━━━━━━━━ 5s 599us/step - accuracy: 0.6857 - loss: 0.5985 - val_accuracy: 0.6880 - val_loss: 0.5921 - learning_rate: 1.1889e-04
Epoch 6/100
7696/7696 ━━━━━━━━━━━━━━━━━━━━ 4s 557us/step - accuracy: 0.6855 - loss: 0.5978 - val_accuracy: 0.6882 - val_loss: 0.5911 - learning_rate: 1.1889e-04
Epoch 7/100
7696/7696 ━━━━━━━━━━━━